<a href="https://colab.research.google.com/github/christamadani/Assignment3/blob/main/Assignment_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#####################################################################################################################
#   Assignment 3: K-means unsupervised learning algorithm
#   Christopher Shatley and Christa Madani
#####################################################################################################################
import copy
import pandas as pd
import random
from tabulate import tabulate


df = pd.read_csv('https://raw.githubusercontent.com/christamadani/Assignment3/main/bbchealth.txt', header=None,
                 delimiter="|")
df.columns = ['a', 'b', 'Line']
df.drop('a', inplace=True, axis=1)
df.drop('b', inplace=True, axis=1)
df = df.replace('#', '', regex=True)
df = df.replace('@\w+', '', regex=True)
df = df.replace('((http)(s?)(:\/\/)(.*)(\.)([\S]*))', '', regex=True)
df['Line'] = df['Line'].str.lower()
df.astype('string')


def k_means_cluster(f_k, f_df, f_r):

    # Create the init centroids
    full_set = set()
    for name, val in f_df.itertuples():
        # Split the text
        text_set = set(val.split())
        full_set = full_set.union(text_set)

    # List of centroids
    centroids = []
    centroids_new = []
    centroids_tweet_count = []

    # We can also just split it by k, we are getting randoms of each point
    for x in range(f_k + 1):

        full_set_list = list(full_set)
        subset = set()
        # Size of random drop
        centroid_size = int((full_set_list.__len__()) / (f_k + 1))

        # Randomly shuffle the list?
        if f_r:
            random.shuffle(full_set_list)

        # Take set of data from start point to end point.
        for word in full_set_list[x * centroid_size: (full_set_list.__len__() - (f_k - x) * centroid_size)]:
            subset = subset.union([word])

        # print(subset)
        centroids.append(copy.deepcopy(subset))
        subset.clear()

    # print(centroids)
    # print()

    # de-facto adjacency matrix.
    centroid_pos_cur = [[0 for n1 in range(f_df.size)] for n2 in range(f_k + 1)]
    centroid_item_cur = [0 for n1 in range(f_df.size)]
    centroid_item_old = [1 for n1 in range(f_df.size)]

    # Till converge 10 centroids, and
    centroid_idx = 0
    item_idx = int(0)

    while centroid_item_cur != centroid_item_old:

        # Reset in case
        centroids_tweet_count.clear()
        centroids_tweet_count = [0] * (f_k + 1)


        # Update previous old
        centroid_item_old = copy.deepcopy(centroid_item_cur)

        # For each item
        for name, val in f_df.itertuples():

            # Check each distance in the allocated centroid if | Calculate distance to the centroid
            current_min = 100000
            for c_m in centroids:

                # Split the text
                text_set = set(val.split())
                # Get the distance
                centroid_pos_cur[centroid_idx][item_idx] = distance(text_set, c_m)

                if centroid_pos_cur[centroid_idx][item_idx] < current_min:
                    current_min = centroid_pos_cur[centroid_idx][item_idx]
                    centroid_item_cur[item_idx] = centroid_idx
                    # print(centroid_idx)
                    # print(current_min)

                centroid_idx = centroid_idx + 1

            centroid_idx = 0
            item_idx = item_idx + 1

        # Update the centroid centers.
        centroid_update = set()
        item_idx = 0

        # For each centroid
        for cent in range(f_k + 1):

            # For each item
            for name, val in f_df.itertuples():
                # Split the text to create new center
                # print('closest centroid ' + str(centroid_item_cur[item_idx]) + 'current centroid ' + str(cent))
                if centroid_item_cur[item_idx] == cent:
                    text_set = set(val.split())
                    # Here actual distance.
                    centroid_update = centroid_update.union(text_set)
                    # Increment the tweet count
                    #print(centroid_idx)
                    tval = centroids_tweet_count[centroid_idx] + 1
                    centroids_tweet_count[centroid_idx] = tval
                item_idx = item_idx + 1

            centroid_idx = centroid_idx + 1
            item_idx = 0
            centroids_new.append(copy.deepcopy(centroid_update))
            centroid_update.clear()

        centroid_idx = 0
        centroids.clear()
        centroids = copy.deepcopy(centroids_new)
        centroids_new.clear()

        # print(centroid_item_cur)
        # print(centroid_item_old)

    SSE = sse(f_df, centroids, centroid_item_cur)
    return centroids, SSE, f_k + 1, centroids_tweet_count


def sse(f_df, centroids, centroid_item_cur):
    dsqu_list = []
    centroid_idx = 0
    item_idx = 0
    item_idx2 = 0
    f_df2 = copy.deepcopy(f_df)

    # Compare all points in a centroid to its neighbors, then div by the size of the centroid
    # We do this becase we don't have a true center of a centroid, and we need the metric of how
    # Tightly it is clustered
    for c_m in centroids:
        for name, val in f_df.itertuples():
            if centroid_idx == centroid_item_cur[item_idx]:
                for name2, val2 in f_df2.itertuples():
                    if centroid_idx == centroid_item_cur[item_idx2]:
                        text_set = set(val.split())
                        text_set2 = set(val2.split())
                        # print("Item Index " + item_idx + " Text set: " + text_set + " Centroid: " + c_m)
                        # Get the distance
                        # dsqu_list.append(distance(text_set, c_m) ** 2)
                        dsqu_list.append((distance(text_set, text_set2) ** 2) / len(c_m))
                    item_idx2 = item_idx2 + 1
                item_idx2 = 0
            item_idx = item_idx + 1
        item_idx = 0
        centroid_idx = centroid_idx + 1

    # print(len(dsqu_list))
    # print(SSE)

    SSE = sum(dsqu_list)
    return SSE


# Abstract distance calculation (I want to make this be variant.) Add param to change distance measure to EUCLIDEAN WOO.
def distance(set1, set2):
    l_u, l_i = jaccard_load(set1, set2)
    l_d = jaccard(l_u, l_i)
    return l_d


# Jaccard Load sets
def jaccard_load(set1, set2):
    l_u = set1.union(set2)
    l_i = set1.intersection(set2)
    return l_u, l_i


# Jaccard calculation
def jaccard(u, i):
    IntersectionLen = len(i)
    UnionLen = len(u)
    return round(1 - (int(IntersectionLen) / UnionLen), 4)


# Run the K-means sim
if __name__ == "__main__":
    head = ["Value of k", "SSE", "Size of each cluster"]
    data = []
    cent_data = []
    for i in range(0, 15, 2):
        # Clusters starting from 1
        f_centroid, f_sse, f_k, f_centroid_tweet = k_means_cluster(i, df, True)
        c_count = 1
        cent_str = ''
        cent_data.clear()
        for l_c in f_centroid_tweet:
            cent_data.append(str(c_count) + " : " + str(l_c) + " tweets")
            c_count = c_count + 1
        for line in cent_data:
            cent_str = cent_str + str(line) + "\n"
        data.append((str(f_k), str(f_sse), cent_str))
        # print(tabulate(data, headers=head, tablefmt="grid"))
    print(tabulate(data, headers=head, tablefmt="grid"))


+--------------+---------+------------------------+
|   Value of k |     SSE | Size of each cluster   |
+==============+=========+========================+
|            1 | 2463.74 | 1 : 3929 tweets        |
+--------------+---------+------------------------+
|            3 | 1876.31 | 1 : 1194 tweets        |
|              |         | 2 : 1208 tweets        |
|              |         | 3 : 1527 tweets        |
+--------------+---------+------------------------+
|            5 | 1757.11 | 1 : 674 tweets         |
|              |         | 2 : 785 tweets         |
|              |         | 3 : 753 tweets         |
|              |         | 4 : 892 tweets         |
|              |         | 5 : 825 tweets         |
+--------------+---------+------------------------+
|            7 | 1680.11 | 1 : 497 tweets         |
|              |         | 2 : 612 tweets         |
|              |         | 3 : 587 tweets         |
|              |         | 4 : 648 tweets         |
|           